In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
# Create an engine to a database file called `hawaii.sqlite`

engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
# Create a connection to the engine called `conn`
conn = engine.connect()


In [4]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column

Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
class Station(Base):
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude= Column(Float)
    elevation = Column(Float)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [5]:
# Use `create_all` to create the  tables in the database
### BEGIN SOLUTION
Base.metadata.create_all(engine)
### END SOLUTION

In [6]:
import pandas as pd

#read CSv's and make them into dataframes
clean_station = "./station_clean.csv"
clean_measurement = "./measurement_clean.csv"
station_clean_df = pd.read_csv(clean_station)
measurement_clean_df= pd.read_csv(clean_measurement)

In [7]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
stations = station_clean_df.to_dict(orient='records')
measurements = measurement_clean_df.to_dict(orient='records')

In [8]:
#Where does the Unnamed come from?

stations[0]

{'Unnamed: 0': 0,
 'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [9]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [10]:
# Save the reference to the `station` table as a variable called `table`
table_station = sqlalchemy.Table('station', metadata, autoload=True)
table_measure = sqlalchemy.Table('measurement', metadata, autoload=True)


In [11]:
# Use `table.delete()` to remove any existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.

conn.execute(table_station.delete())
conn.execute(table_measure.delete())




In [12]:
# Use `table.insert()` to insert the data into the table

conn.execute(table_station.insert(), stations)
conn.execute(table_measure.insert(), measurements)


In [13]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]